In [665]:
import pandas as pd
import datetime 
import numpy as np
import math

In [666]:
df=pd.read_csv('allied2.csv')


In [667]:
my_list = df.columns.values.tolist()

In [668]:
for i in my_list: 
    print(i)

Agency Name
Employee Last Name
Employee First Name
Participant Last Name
Participant First Name
Participant Timezone
Scheduled Visit Start
Scheduled Visit End
Actual Visit Start
Actual Visit End
Notes
Status 


In [669]:
df['Scheduled Visit End'] = df['Scheduled Visit End'].astype('datetime64[ns]')
df['Scheduled Visit Start'] = df['Scheduled Visit Start'].astype('datetime64[ns]')
df["Scheduled Visit Start"].dtype

dtype('<M8[ns]')

In [670]:
df['SVS'] = df['Scheduled Visit Start']
df['SVE'] = df['Scheduled Visit End']
df["SVS"].dtype

dtype('<M8[ns]')

In [671]:
df=df.sort_values(by=['Scheduled Visit Start','Participant First Name'])

In [672]:
df['DayOftheWeek'] = df['Scheduled Visit Start'].dt.dayofweek
df["WeekOfyear"] = df['Scheduled Visit Start'].dt.isocalendar().week
df["Day"] = df['Scheduled Visit Start'].dt.day
df['Month'] = df['Scheduled Visit Start'].dt.month
df['Year'] = df['Scheduled Visit Start'].dt.year


In [673]:
df['hourDigit'] = df['Scheduled Visit Start'].dt.hour
df['Actual Visit Start'] = df['Actual Visit Start'].astype('datetime64[ns]')
df['Actual Visit End'] = df['Actual Visit End'].astype('datetime64[ns]')
df['SchedMin']=df['Scheduled Visit End']-df['Scheduled Visit Start']
df["SchedMin"] = df["SchedMin"].to_numpy()/np.timedelta64(1, 'm')
df['AcctMIn']=df['Actual Visit End']-df['Actual Visit Start']
df['AcctMIn'] = df['AcctMIn'].to_numpy()/np.timedelta64(1, 'm')
df['StartDif']=df['Actual Visit Start']-df['Scheduled Visit Start']
df["StartDif"] = df["StartDif"].to_numpy()/np.timedelta64(1, 'm')
df['EndDif']=(df['Actual Visit End']-df['Scheduled Visit End'])
df["EndDif"] = df["EndDif"].to_numpy()/np.timedelta64(1, 'm')
df['HoursDif']=(df['AcctMIn']-df['SchedMin'])/60.00
df['Untis']=df['SchedMin']/15
df['Difference']=df['AcctMIn']-df['SchedMin']

In [674]:
df=df.sort_values(by=['Scheduled Visit Start'])
df['Code']=+df['hourDigit'].map(str)+"_"+df['Employee First Name'].map(str)+"_"+df['Participant First Name'].map(str)
df=df.sort_values(by=['Code','Scheduled Visit Start'])

In [675]:
df['Timelaps']=df["Scheduled Visit Start"].diff()
df['Timelaps'].diff().apply(lambda x: x/np.timedelta64(1, 'D')).fillna(0).astype('int64')
df['Timelaps'] = df['Timelaps'].to_numpy()/np.timedelta64(1, 'D')
def truncf(num):
    if num == None:
        num1=0
    else:
        num1=float(num)
    return num1
df['Timelaps'] = df['Timelaps'].apply(truncf).fillna(0)
def trunci(num):
    num1=int(num)
    return num1
df['Timelaps'] = df['Timelaps'].apply(trunci)
df.loc[df['Timelaps'] <=1,"Split"]=0
df.loc[df['Timelaps'] >1,"Split"]=1
df["Split"]=df["Split"].astype(int)
df['split_csum'] = df.groupby(['WeekOfyear'])['Split'].cumsum()

In [676]:
dfbyparticapant=df.groupby(['Participant First Name','hourDigit']).sum()
dfbyparticapant

Notes  DayOftheWeek  WeekOfyear   Day  \
Participant First Name hourDigit                                          
Nouri                  5            0.0           222        2007  1675   
                       8            0.0           241         919   667   
Zainab                 13           0.0           213         723   620   
                       17           0.0           230        2174  1798   

                                  Month    Year  SchedMin       AcctMIn  \
Participant First Name hourDigit                                          
Nouri                  5            511  218350   29820.0  52992.150000   
                       8            213   88957   13200.0  17500.133333   
Zainab                 13           176   78850   17640.0  24322.166667   
                       17           551  230479   27360.0  47655.050000   

                                    StartDif        EndDif    HoursDif  \
Participant First Name hourDigit                                         
Nouri                  5          -47.450000  23124.700000  386.202500   
                       8           44.166667   4344.300000   71.668889   
Zainab                 13         106.750000   6788.916667  111.369444   
                       17         472.716667  20767.766667  338.250833   

                                   Untis    Difference  Timelaps  Split  \
Participant First Name hourDigit                                          
Nouri                  5          1988.0  23172.150000         0     29   
                       8           880.0   4300.133333         2     22   
Zainab                 13         1176.0   6682.166667       168     21   
                       17         1824.0  20295.050000         0     26   

                                  split_csum  
Participant First Name hourDigit              
Nouri                  5                 312  
                       8                 176  
Zainab                 13                 37  
                       17                210

In [677]:
def wholeuntis(mins):
    remainder= (mins%60)
    hours=(mins-remainder)/60
    if remainder in range(6, 11):
        units = 1
    elif remainder in range(12, 27):
        units = 2
    elif remainder in range(28, 43):
        units = 3
    elif remainder in range(44, 60):
        units = 4
    else:
        units = 0
    return (hours*4)+units
df['Whnts']=df['Difference'].apply(wholeuntis).astype(int)

In [678]:
df['RolingSplit']=df['Split'].rolling(1).mean()

In [679]:
df['sftWOY']=df["WeekOfyear"].shift(1)

In [680]:
df['Splitindc']=df["WeekOfyear"]-df['sftWOY']+df['Split']

In [681]:
df['Count']=1

In [682]:
df['Grouping']=df["Month"].map(str)+"_"+df["WeekOfyear"].map(str)+"_"+df["split_csum"].map(str)

In [683]:
df['MaxSVS'] = df.groupby('Grouping')["Day"].transform('max')
df['MinSVS'] = df.groupby('Grouping')["Day"].transform('min')

In [684]:
df1=df

In [685]:
df['StartDate']=df["Month"].map(str)+"/"+df["MinSVS"].map(str)+"/"+df["Year"].map(str)
df['EndDate']=df["Month"].map(str)+"/"+df["MaxSVS"].map(str)+"/"+df["Year"].map(str)

In [686]:
g1=df1.groupby(['Code','Grouping','StartDate','EndDate'],as_index=False).sum()
g1.add_suffix('_Count').reset_index()
g1

,Code,Grouping,StartDate,EndDate,Notes,DayOftheWeek,WeekOfyear,Day,Month,Year,...,Timelaps,Split,split_csum,Whnts,RolingSplit,sftWOY,Splitindc,Count,MaxSVS,MinSVS
0,13_Maitham_Zainab,11_45_0,11/13/2021,11/14/2021,0.0,11,90,27,22,4042,...,1,0,0,0,0.0,45,0,2,28,26
1,13_Maitham_Zainab,12_48_1,12/4/2021,12/4/2021,0.0,5,48,4,12,2021,...,20,1,1,0,1.0,45,4,1,4,4
2,13_Maitham_Zainab,12_49_1,12/11/2021,12/12/2021,0.0,11,98,23,24,4042,...,8,1,2,0,1.0,97,2,2,24,22
3,13_Maitham_Zainab,12_50_1,12/18/2021,12/19/2021,0.0,11,100,37,24,4042,...,7,1,2,32,1.0,99,2,2,38,36
4,13_Maitham_Zainab,12_51_1,12/25/2021,12/25/2021,0.0,5,51,25,12,2021,...,6,1,1,-4,1.0,50,2,1,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,8_Maitham_Nouri,4_13_4,4/2/2022,4/3/2022,0.0,11,26,5,8,4044,...,7,1,8,72,1.0,25,2,2,6,4
104,8_Maitham_Nouri,4_14_4,4/9/2022,4/10/2022,0.0,11,28,19,8,4044,...,7,1,8,0,1.0,27,2,2,20,18
105,8_Maitham_Nouri,4_15_4,4/16/2022,4/17/2022,0.0,11,30,33,8,4044,...,7,1,8,-4,1.0,29,2,2,34,32
106,8_Maitham_Nouri,4_16_4,4/23/2022,4/24/2022,0.0,11,32,47,8,4044,...,7,1,8,-4,1.0,31,2,2,48,46


In [687]:
g1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Code          108 non-null    object 
 1   Grouping      108 non-null    object 
 2   StartDate     108 non-null    object 
 3   EndDate       108 non-null    object 
 4   Notes         108 non-null    float64
 5   DayOftheWeek  108 non-null    int64  
 6   WeekOfyear    108 non-null    Int64  
 7   Day           108 non-null    int64  
 8   Month         108 non-null    int64  
 9   Year          108 non-null    int64  
 10  hourDigit     108 non-null    int64  
 11  SchedMin      108 non-null    float64
 12  AcctMIn       108 non-null    float64
 13  StartDif      108 non-null    float64
 14  EndDif        108 non-null    float64
 15  HoursDif      108 non-null    float64
 16  Untis         108 non-null    float64
 17  Difference    108 non-null    float64
 18  Timelaps      108 non-null    

In [688]:
g2=g1[['Code','Grouping','StartDate','EndDate','Untis']]

In [689]:
g2

,Code,Grouping,StartDate,EndDate,Untis
0,13_Maitham_Zainab,11_45_0,11/13/2021,11/14/2021,60.0
1,13_Maitham_Zainab,12_48_1,12/4/2021,12/4/2021,32.0
2,13_Maitham_Zainab,12_49_1,12/11/2021,12/12/2021,60.0
3,13_Maitham_Zainab,12_50_1,12/18/2021,12/19/2021,60.0
4,13_Maitham_Zainab,12_51_1,12/25/2021,12/25/2021,32.0
...,...,...,...,...,...
103,8_Maitham_Nouri,4_13_4,4/2/2022,4/3/2022,40.0
104,8_Maitham_Nouri,4_14_4,4/9/2022,4/10/2022,40.0
105,8_Maitham_Nouri,4_15_4,4/16/2022,4/17/2022,40.0
106,8_Maitham_Nouri,4_16_4,4/23/2022,4/24/2022,40.0


In [690]:
g2.to_csv('ouput2.csv')

In [691]:
df.to_csv('output.csv')

PermissionError: [Errno 13] Permission denied: 'output.csv'